In [27]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import numpy as np

from ast import literal_eval

In [12]:
sentiment_pipeline = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [13]:
data = ["I love you", "I am running a marathon without shoes"]
sentiment_pipeline(data)

[{'label': 'POSITIVE', 'score': 0.9998656511306763},
 {'label': 'NEGATIVE', 'score': 0.9845187664031982}]

In [16]:
df_billboard = pd.read_csv('data/df_billboard_0_100.csv')
df_billboard.head()

,song,artist,url,lyrics,lyrics_text
0,Blinding Lights,The Weeknd,https://genius.com/The-weeknd-blinding-lights-...,"[('[Intro]', 'yeah'), ('[Verse 1]', ""i've been...",yeah i've been tryna call i've been on my own ...
1,Radioactive,Imagine Dragons,https://genius.com/Imagine-dragons-radioactive...,"[('[Intro]', 'whoa, oh-oh whoa, oh-oh whoa, oh...","whoa, oh-oh whoa, oh-oh whoa, oh-oh whoa i'm w..."
2,Sail,AWOLNATION,https://genius.com/Awolnation-sail-lyrics,"[('[Intro]', 'sail'), ('[Verse 1]', 'this is h...",sail this is how i show my love i made it in m...
3,I'm Yours,Jason Mraz,https://genius.com/Jason-mraz-im-yours-lyrics,"[('[Verse 1]', ""well you done done me in; you ...",well you done done me in; you bet i felt it i ...
4,How Do I Live,LeAnn Rimes,https://genius.com/Leann-rimes-how-do-i-live-l...,"[('[Verse 1]', 'how do i get through one night...",how do i get through one night without you? if...


In [21]:
df_billboard['lyrics'] = df_billboard['lyrics'].map(literal_eval)

In [26]:
lyrics = df_billboard['lyrics'].to_list()
lyrics[0]

[('[Intro]', 'yeah'),
 ('[Verse 1]',
  "i've been tryna call i've been on my own for long enough maybe you can show me\u2005how\u2005to love, maybe i'm\u2005going through withdrawals you don't even have\u2005to do too much you can turn me on with just a touch, baby"),
 ('[Pre-Chorus]',
  "i look around and sin city's cold and empty (oh) no one's around to judge me (oh) i can't see clearly when you're gone"),
 ('[Chorus]',
  "i said, ooh, i'm blinded by the lights no, i can't sleep until i feel your touch i said, ooh, i'm drowning in the night oh, when i'm like this, you're the one i trust hey, hey, hey"),
 ('[Verse 2]',
  "i'm running out of time 'cause i can see the sun light up the sky so i hit the road in overdrive, baby, oh"),
 ('[Pre-Chorus]',
  "the city's cold and empty (oh) no one's around to judge me (oh) i can't see clearly when you're gone"),
 ('[Chorus]',
  "i said, ooh, i'm blinded by the lights no, i can't sleep until i feel your touch i said, ooh, i'm drowning in the nig

In [30]:
song_parts, text = list(zip(*lyrics[0]))
text

('yeah',
 "i've been tryna call i've been on my own for long enough maybe you can show me\u2005how\u2005to love, maybe i'm\u2005going through withdrawals you don't even have\u2005to do too much you can turn me on with just a touch, baby",
 "i look around and sin city's cold and empty (oh) no one's around to judge me (oh) i can't see clearly when you're gone",
 "i said, ooh, i'm blinded by the lights no, i can't sleep until i feel your touch i said, ooh, i'm drowning in the night oh, when i'm like this, you're the one i trust hey, hey, hey",
 "i'm running out of time 'cause i can see the sun light up the sky so i hit the road in overdrive, baby, oh",
 "the city's cold and empty (oh) no one's around to judge me (oh) i can't see clearly when you're gone",
 "i said, ooh, i'm blinded by the lights no, i can't sleep until i feel your touch i said, ooh, i'm drowning in the night oh, when i'm like this, you're the one i trust",
 "i'm just calling back to let you know (back to let you know) i c

In [78]:
from transformers import pipeline

sentiment_pipeline = pipeline("sentiment-analysis")

def get_positivity_scores(text):
    raw_scores = sentiment_pipeline(list(text))
    scores = [raw_score['score'] if raw_score['label'] == 'POSITIVE' else -raw_score['score'] for raw_score in raw_scores]
    return scores


def append_positivity(all_lyrics):
    positivity_list = []
    for lyrics in all_lyrics:
        parts_text = list(zip(*lyrics))
        if parts_text:
            positivity = list(zip(parts_text[0], get_positivity_scores(text)))
            positivity_list.append(positivity)
        else:
            positivity_list.append([])
    df_billboard['positivity'] = positivity_list

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [79]:
append_positivity(lyrics)

In [81]:
df_billboard.to_csv('data/positivity.csv')